In [63]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
sns.set_palette("pastel")
from multiprocessing import Pool
from tqdm import tqdm
import statsmodels.api as sm
from datetime import datetime
import mplfinance as mpf
from functools import partial

In [64]:
def daily_K_line(event,n,open,close,high,low):
    # 日均事件K线
    T0open=event*open  # 以T0open作为基准
    pre_close = event * close.shift(axis = 1)
    data=pd.DataFrame(columns=['T'+str(j) for j in range(n)])
    for j in range(n):
        _open=(event*open.shift(-j,axis=1))/pre_close
        _close=(event*close.shift(-j,axis=1))/pre_close
        _low=(event*low.shift(-j,axis=1))/pre_close
        _high=(event*high.shift(-j,axis=1))/pre_close
        lst=[_open.mean().mean(),_close.mean().mean(),_low.mean().mean(),_high.mean().mean()]
        data['T'+str(j)]=lst
    data.index=['open','close','low','high']
    data=data.T
    data.index=pd.to_datetime(['20200101','20200102','20200103','20200104','20200105'])
    # mpf.plot(data,type='candle',style='default')
    return data

In [65]:
def setg(x):
    if x<1:
        return 0
    elif x<2:
        return 1
    elif x<5:
        return 2
    elif x<15:
        return 3
    elif x<30:
        return 4
    elif x<60:
        return 5
    elif x<120:
        return 6
    elif x<180:
        return 7
    elif x<240:
        return 8

In [66]:
stock_index = ff.read('一字板').index
time_index = ff.read('一字板').columns
up = ff.read('up').loc[stock_index,time_index].copy()
# open = ff.read('open').loc[stock_index,time_index].copy()
close = ff.read('close').loc[stock_index,time_index].copy()
high = ff.read('high').loc[stock_index,time_index].copy()
low = ff.read('low').loc[stock_index,time_index].copy()
pre_close = ff.read('pre_close').loc[stock_index,time_index].copy()
up_limit = pd.DataFrame(ff.read('up_limit').copy(),index = stock_index,columns = time_index) * ff.filter0.loc[stock_index,time_index]

In [67]:
up_filter = (high == up_limit)
dict = {}
for i in up_filter.sum(axis = 1)[up_filter.sum(axis = 1)>0].index:
    stock = up_filter.loc[i,:]
    dict[i] = list(stock[stock == True].index)

In [68]:
def up_first(stock):
    first = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        start_1 = np.argwhere((data.loc[:,'high'] == daily_up_limit).values)[0][0]
        first.append(start_1)
    return first
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_first,dict.keys()),total = len(dict.keys())))
up_first_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_first_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4423/4423 [05:07<00:00, 14.39it/s]


In [69]:
factor_names1=['closer', 'openr', 'lowr', 'highr', 'ocret', 'lcret', 'hcret', 'ohret', 'ret5', 'ret10', 'ret20', 'ret30', 'excess5', 'excess10', 'excess20', 'excess30', 'ma5', 'bias5', 'ma10',
               'bias10', 'ma15', 'bias15', 'ma20', 'bias20', 'vwap_ret', 'vwaph_ret', 'll5', 'll15', 'll20', 'll30', 'a0', 'a5', 'a15', 'a30', 'ah5', 'ah10', 'ah15', 'std', 'amt_std', 'vol_std', 
               'amt_mean', 'vol_mean', 'turnover', 'turnover_mean', 'turnover_std', 'turnover_skew', 'turnoverf', 'turnoverf_mean', 'turnoverf_std', 'turnoverf_skew', 'mkv', 'l_mkv', 'lhret', 
               'STR', 'MTR', 'SMTR', 'ILLIQ','close','high','low']
factor_names2=['ARRP', 'ARRP_5d_20mean', 'AmpMod', 'BVol', 'CDPDVP', 'ILLIQ', 'IVoldeCorr', 'IntraDayMom20', 'MTR', 'RCP', 'SMTR', 'SPS', 'STR', 'a30', 
               'afternoon_ratio_1', 'ah15', 'amt', 'amt_ma20', 'amt_mean', 'amt_score0', 'amt_score1', 'amt_score2', 'amt_std', 'amt_std20', 'bias10', 'bias15', 'bias20', 'bias5', 
               'cross_std10', 'cross_std10adj', 'ctrade', 'excess10', 'excess20', 'excess30', 'excess5', 'growth_a', 'growth_b', 'growth_c', 'h_rankamt_corr_10', 'h_rankv_corr_10', 'h_rankv_corr_36', 
               'hcret', 'idiov', 'intro_high80_corr', 'ivr', 'l_mkv', 'mkv', 'mom1', 'mom_spring', 'nl_mkv', 'nl_mom', 'open0931ratio', 'open5ratio_ma10vol', 'rank_2_corr_hv10', 'rank_2_corr_hv20', 'rank_ha_corr_sum', 
               'ret20', 'ret30', 'ret5', 'roc121', 'roc20', 'roc240', 'roc5', 'scr', 'std', 'std20', 'std21', 'std240', 'stddev_cov', 'stddev_cov_right', 'sub', 'sysv', 'tail_ratio_1', 'tailrets0', 'tailrets1', 'tliq', 
               'turn20', 'turnover', 'turnover_mean', 'turnover_scale_z', 'turnover_score_ts_mean', 'turnover_score_ts_std', 'turnover_skew', 'turnover_std', 'turnoverf', 'turnoverf_mean', 'turnoverf_skew', 'turnoverf_std', 
               'up_KURS', 'up_plus_down_KURS', 'vol', 'vol_mean', 'vol_std', 'vol_std20', 'volroc_skew', '滞后日内量价corr']
names= list(np.unique(factor_names1+factor_names2))

In [70]:
def five_quantile(x):
    result = x.copy()
    x[(result<=result.quantile(0.2))] = 0
    x[(result>result.quantile(0.2))*(result<=result.quantile(0.4))] = 1
    x[(result>result.quantile(0.4))*(result<=result.quantile(0.6))] = 2
    x[(result>result.quantile(0.6))*(result<=result.quantile(0.8))] = 3
    x[(result>result.quantile(0.8))] = 4
    return x

In [9]:
shape_data =  (ff.read('单次涨停').copy().replace({np.nan:0}) + ff.read('多次涨停').copy().replace({np.nan:0}) + ff.read('炸板股').copy().replace({np.nan:0})).replace({0:np.nan}) * (ff.read('10cm').copy())
feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())

In [10]:
def factor_touch(factor_name):
    stat = pd.DataFrame(columns = ['因子','因子分组','首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
    factor = pd.DataFrame(ff.read(factor_name),index = stock_index,columns=time_index).copy().apply(five_quantile).shift(axis = 1)
    for j in range(5):
        factor_ = (factor == j).replace({False:np.nan,True:1.0})
        for i in range(0,9):
            num_ = (feature_data == i).replace({False:np.nan,True:1.0})
            event_ =  shape_data * num_ * factor_
            data = daily_K_line(event_,5,open_,close_,high_,low_)
            stat.loc[len(stat.index)] = [factor_name,j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]
    return stat

In [11]:
open_ = (ff.read('open') * ff.read('post')).loc[stock_index,time_index].copy()
close_ = (ff.read('close')* ff.read('post')).loc[stock_index,time_index].copy()
high_ = (ff.read('high')* ff.read('post')).loc[stock_index,time_index].copy()
low_ = (ff.read('low')* ff.read('post')).loc[stock_index,time_index].copy()

In [12]:
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(factor_touch,names),total = len(names)))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [11:32<00:00,  5.63s/it]


In [13]:
result = pd.concat(res_lst)

In [18]:
result = result.sort_values(by = '合计收益',ascending = False)
result

,因子,因子分组,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
19,ll20,2,1,0.001171,0.0,0.001171,0.100295,0.014749,0.109145,0.015044
10,mom_spring,1,1,0.189696,0.0,0.176815,0.086273,0.026668,0.016791,0.012940
11,bias20,1,2,0.346604,0.0,0.289227,0.089690,0.022661,0.014996,0.012351
2,excess30,0,2,0.300937,0.0,0.245902,0.088901,0.019869,0.014314,0.008770
2,ret30,0,2,0.300937,0.0,0.245902,0.088901,0.019869,0.014314,0.008770
...,...,...,...,...,...,...,...,...,...,...
22,ll5,2,4,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN
23,ll5,2,5,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN
24,ll5,2,6,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN
25,ll5,2,7,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN


In [19]:
import pickle 
with open('/home/wangs/data/feature/因子_首次触板时间分组.pk','wb') as f:
    pickle.dump(result,f)

前一天涨停一字板 + 第二天触板时刻 + 第二天开盘未涨停

In [53]:
stat_up = pd.DataFrame(columns = ['首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
shape_data =  ff.read('一字板').loc[stock_index,time_index].copy().shift(axis = 1)
feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
open_up = (ff.read('open').loc[stock_index,time_index].copy() == up_limit).replace({False:1.0,True:np.nan})
for i in range(0,9):
    num_ = (feature_data == i).replace({False:np.nan,True:1.0})
    event_ =  shape_data * num_ * open_up 
    data = daily_K_line(event_,5,open_,close_,high_,low_)
    stat_up.loc[len(stat_up.index)] = [i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

In [54]:
stat_up.sort_values(by = '合计收益',ascending = False)

,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
0,0.0,0.147541,0.0,0.139344,0.083166,0.015746,0.007495,-0.001088
2,2.0,0.104215,0.0,0.100703,0.079397,0.005017,-0.006096,-0.015586
6,6.0,0.039813,0.0,0.038642,0.083123,-0.004904,0.012462,-0.021781
4,4.0,0.049180,0.0,0.048009,0.074280,0.003864,0.018190,-0.021856
8,8.0,0.026932,0.0,0.025761,0.094407,-0.021210,-0.017108,-0.026804
7,7.0,0.039813,0.0,0.037471,0.077985,-0.005081,0.000745,-0.027096
3,3.0,0.125293,0.0,0.112412,0.076046,-0.006120,0.017605,-0.030074
1,1.0,0.073770,0.0,0.069087,0.075052,-0.007286,0.007802,-0.032234
5,5.0,0.037471,0.0,0.037471,0.068165,-0.018125,0.002340,-0.049961


前一天涨停不包括一字板 + 第二天触板时刻

In [51]:
stat = pd.DataFrame(columns = ['首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
shape_data =  (ff.read('一字板').loc[stock_index,time_index].copy().replace({np.nan:1,1.0:np.nan}) * ff.read('up').copy()).shift(axis = 1)
feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
for i in range(0,9):
    num_ = (feature_data == i).replace({False:np.nan,True:1.0})
    event_ =  shape_data * num_ 
    data = daily_K_line(event_,5,open_,close_,high_,low_)
    stat.loc[len(stat.index)] = [i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

In [52]:
stat.sort_values(by = '合计收益',ascending = False)

,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
0,0.0,1.086068,0.0,0.241486,0.075495,0.026497,0.019089,0.001992
3,3.0,0.417337,0.0,0.195046,0.076012,0.011002,0.011631,-0.012986
1,1.0,0.216099,0.0,0.143344,0.070603,0.015755,0.012213,-0.013642
6,6.0,0.256966,0.0,0.146130,0.081271,0.002559,0.010113,-0.016170
7,7.0,0.229102,0.0,0.135294,0.082502,0.000517,0.007051,-0.016980
2,2.0,0.289474,0.0,0.166873,0.070333,0.011433,0.008037,-0.018234
4,4.0,0.289783,0.0,0.157895,0.073525,0.006609,0.007533,-0.019866
5,5.0,0.275851,0.0,0.157895,0.076113,0.003082,0.007864,-0.020806
8,8.0,0.188854,0.0,0.118885,0.087334,-0.009664,-0.010449,-0.022331


因子 + 当天的开盘涨幅 + 首次触板时刻 + 前一天涨停不包括一字板

In [71]:
open_ret = ff.read('open').loc[stock_index,time_index].copy() /pre_close
open_ret = open_ret.apply(five_quantile) #当天的开盘涨幅分组

In [72]:
shape_data =  (ff.read('一字板').loc[stock_index,time_index].copy().replace({np.nan:1,1.0:np.nan}) * ff.read('up').copy()).shift(axis = 1) #前一天涨停不包括一字板
feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())

In [73]:
def factor_touch(factor_name):
    stat = pd.DataFrame(columns = ['因子','因子分组','首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
    factor = pd.DataFrame(ff.read(factor_name),index = stock_index,columns=time_index).copy().apply(five_quantile).shift(axis = 1)
    for j in range(5):
        factor_ = (factor == j).replace({False:np.nan,True:1.0})
        for i in range(0,9):
            num_ = (feature_data == i).replace({False:np.nan,True:1.0})
            event_ =  shape_data * num_ * factor_
            data = daily_K_line(event_,5,open_,close_,high_,low_)
            stat.loc[len(stat.index)] = [factor_name,j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]
    return stat
with Pool(24) as p:
    res_lst2 = list(tqdm(p.imap(factor_touch,names),total = len(names)))
result2 = pd.concat(res_lst2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [46:44<00:00, 22.80s/it]


In [74]:
for j in range(5):
    open_ret_ = (open_ret == j).replace({False:np.nan,True:1.0})
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  shape_data * num_ * open_ret_
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        result2.loc[len(result2.index)] = ['开盘涨跌幅',j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]